In [30]:
import os
import pandas as pd

### Get real script lines for each character

In [31]:
def get_str_script_from_df(df):
    # convert df into a string episode script
    df = df.iloc[:, 2:].fillna('<scene>')
    
    # Concatenate the remaining columns into a single string for each row
    df['formatted_string'] = df.apply(lambda row: ': '.join(row.astype(str)), axis=1)
    
    # Combine all rows into a single string separated by line breaks
    final_string = '\n'.join(df['formatted_string'])
    return final_string.replace("\'", "`").lower()

In [73]:
SEASONS = [8,9,10,11,12,13,14,15]
CHARACTERS = ["stan", "kyle", "cartman", "butters"]

cartman_data = []
stan_data = []
kyle_data = []
butters_data = []

for SEASON in SEASONS:
    # get all .csv files in correct season dir
    episodes = list(filter(lambda file: file[-4:] == ".csv", os.listdir(os.path.join(".", "episodes_csv", f"s{SEASON}"))))
    print(f"Season {SEASON}!")
    for episode in episodes:
        df = pd.read_csv(os.path.join(".", "episodes_csv", f"s{SEASON}", episode), delimiter=';', header=None)
        str_script = get_str_script_from_df(df)

        lines = str_script.split("\n")
        
        for line in lines:
            try:
                split_line = line.split(": ")
                character = split_line[0]
                char_line = split_line[1]
                
                if character not in CHARACTERS:
                    continue
                    
                if character == "cartman":
                    cartman_data.append(char_line)
                elif character == "stan":
                    stan_data.append(char_line)
                elif character == "kyle":
                    kyle_data.append(char_line)
                elif character == "butters":
                    butters_data.append(char_line)
            except:
                continue

Season 8!
Season 9!
Season 10!
Season 11!
Season 12!
Season 13!
Season 14!
Season 15!


### Get generated script lines for each character

In [25]:
def get_eps_from_dir(dirpath: str):
    dirs = list(filter(lambda filename: filename[0] == "2", os.listdir(dirpath)))
    
    cartman = []
    stan = []
    kyle = []
    butters = []
    
    for dir in dirs:
        try:
            file = os.path.join(dirpath, dir, "ep.txt")
            with open(file, "r") as f:
                script = f.read()
        except:
            continue
        lines = script.split("\n")
        for line in lines:
            split_line = line.split(": ")
            if len(split_line) < 2:
                continue
            character = split_line[0]
            char_line = " ".join(split_line[1:])
    
            if character == "cartman":
                cartman.append(char_line)
            elif character == "stan":
                stan.append(char_line)
            elif character == "butters":
                butters.append(char_line)
            elif character == "kyle":
                kyle.append(char_line)
    return " ".join(cartman), " ".join(stan), " ".join(kyle), " ".join(butters)

In [18]:
import re
def preprocess_text(txt: str):
    txt = txt.lower()
    # remove punctuation
    txt = re.sub(r'[^\w\s]','',txt)
    return txt

In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joachimmaksim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [78]:
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize

def find_average_sentence_length(txt: str):
    sentences = sent_tokenize(txt) 
    sentence_lengths = list(map(lambda s: len(s), sentences))
    return sum(sentence_lengths) / len(sentence_lengths)

def find_average_word_length(txt: str):
    words = txt.split(" ")
    word_lengths = list(map(lambda w: len(w), words))
    return sum(word_lengths) / len(word_lengths)

def find_rouge_score(generated: str, target: str):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=False)
    return scorer.score(preprocess_text(target), preprocess_text(generated))["rouge1"].precision

In [84]:
# get the various types of datasets
tuned_cartman, tuned_stan, tuned_kyle, tuned_butters = get_eps_from_dir("gen_eps")
untuned_cartman, untuned_stan, untuned_kyle, untuned_butters = get_eps_from_dir("gen_eps_untuned")
untuned_cartman_m, untuned_stan_m, untuned_kyle_m, untuned_butters_m = get_eps_from_dir("gen_eps_untuned_tuned_manager")

real_cartman = " ".join(cartman_data)
real_stan = " ".join(stan_data)
real_kyle = " ".join(kyle_data)
real_butters = " ".join(butters_data)

In [80]:
def compare_sentence_lengths(gen: str, real: str):
    gen_len = find_average_sentence_length(gen)
    gen_real = find_average_sentence_length(real)
    print(f"Sentence length:\tgen / real = \t{gen_len / gen_real}")

def compare_word_lengths(gen: str, real: str):
    gen_len = find_average_word_length(gen)
    gen_real = find_average_word_length(real)
    print(f"Word length:\t\tgen / real =  \t{gen_len / gen_real}")

def compare_rouge(gen: str, real: str):
    rouge = find_rouge_score(gen, real)
    print(f"rouge-1 precision:\t\t\t{rouge}")

In [70]:
def print_char_ratings(char_name: str, tuned, untuned, untuned_m, real):
    print(f"===== \t {char_name} \t=========")
    # === tuned ===
    print("\nTuned")
    compare_sentence_lengths(tuned, real)
    compare_word_lengths(tuned, real)
    compare_rouge(tuned, real)
    
    # === untuned with tuned manager ===
    print("\nUntuned with tuned manager")
    compare_sentence_lengths(untuned_m, real)
    compare_word_lengths(untuned_m, real)
    compare_rouge(untuned_m, real)
    
    # === untuned ===
    print("\nUntuned")
    compare_sentence_lengths(untuned, real)
    compare_word_lengths(untuned, real)
    compare_rouge(untuned, real) 
    print()
    print()
    

In [85]:
print_char_ratings("Cartman", tuned_cartman, untuned_cartman, untuned_cartman_m, real_cartman)
print_char_ratings("Stan", tuned_stan, untuned_stan, untuned_stan_m, real_stan)
print_char_ratings("Kyle", tuned_kyle, untuned_kyle, untuned_kyle_m, real_kyle)
print_char_ratings("Butters", tuned_butters, untuned_butters, untuned_butters_m, real_butters)

===== 	 Cartman 	=========

Tuned
Sentence length:	gen / real = 	1.0931909176419095
Word length:		gen / real =  	1.0163174831817068
rouge-1 precision:			0.8647144948755491

Untuned with tuned manager
Sentence length:	gen / real = 	1.2775439856649602
Word length:		gen / real =  	1.0030567064696565
rouge-1 precision:			0.8818529536761581

Untuned
Sentence length:	gen / real = 	1.3543604513905285
Word length:		gen / real =  	1.002887596010105
rouge-1 precision:			0.8967551622418879


===== 	 Stan 	=========

Tuned
Sentence length:	gen / real = 	0.9330047601018663
Word length:		gen / real =  	1.0333306727280516
rouge-1 precision:			0.8901918976545842

Untuned with tuned manager
Sentence length:	gen / real = 	2.0842729652773797
Word length:		gen / real =  	1.082709450996585
rouge-1 precision:			0.8901098901098901

Untuned
Sentence length:	gen / real = 	1.4975630245218536
Word length:		gen / real =  	1.0162948568702237
rouge-1 precision:			0.8991150442477877


===== 	 Kyle 	=========

Tuned
